In [ ]:
"""
Created on Wed Mar 20 15:07 2024

This script is to make 2D maps of viability limits

@author: Clara Burgard
"""

In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
import seaborn as sns
from matplotlib import cm
import matplotlib as mpl
import cmocean
import glob
from matplotlib.colors import ListedColormap, LinearSegmentedColormap
import os

In [ ]:
sns.set_context('paper')

In [ ]:
%matplotlib qt5

In [ ]:
# make the domain a little smaller to make the computation even more efficient - file isf has already been made smaller at its creation
map_lim = [-3000000,3000000]

READ DATA

In [ ]:
home_path = '/bettik/burgardc/'
plot_path = '/bettik/burgardc/PLOTS/summer_paper_plots/'
outputpath_GL = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/GL_FLUX/'
inputpath_weights = '/bettik/burgardc/DATA/SUMMER_PAPER/processed/ANALYSIS/'
inputpath_atmo = '/bettik/burgardc/DATA/SUMMER_PAPER/raw/TS_SMB_DATA/out/'

In [ ]:
inputpath_mask = home_path+'/DATA/SUMMER_PAPER/interim/ANTARCTICA_IS_MASKS/BedMachine_4km/'
file_isf_orig = xr.open_dataset(inputpath_mask+'BedMachinev2_4km_isf_masks_and_info_and_distance_oneFRIS.nc')
nonnan_Nisf = file_isf_orig['Nisf'].where(np.isfinite(file_isf_orig['front_bot_depth_max']), drop=True).astype(int)
file_isf_nonnan = file_isf_orig.sel(Nisf=nonnan_Nisf)
rignot_isf = file_isf_nonnan.Nisf.where(np.isfinite(file_isf_nonnan['isf_area_rignot']), drop=True)
file_isf = file_isf_nonnan.sel(Nisf=rignot_isf)

In [ ]:
isf_mask = file_isf['ISF_mask']

In [ ]:
file_isf_mask = file_isf['ISF_mask'].where(file_isf['ISF_mask']==file_isf.Nisf).sum('Nisf')

In [ ]:
weight_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined.nc')
weight_2300_file = xr.open_dataset(inputpath_weights + 'bayesian_weights_davison_varying_combined_2300.nc')
file_viability_info = xr.open_dataset(inputpath_weights + 'all_fluxes_br.nc')

PREPARE DATA

In [ ]:
sens_weights = xr.DataArray(data=np.array([0.11,
                                           0.24,
                                           0.03,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.10,
                                           0.24,
                                           0.47,
                                           0.41,
                                           0.12,
                                           0.43,
                                           0.39,
                                           0.05]), dims=['model']).assign_coords({'model': 
                                                                                  ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5',
                                                                                   'CESM2','CESM2-WACCM','CNRM-CM6-1','CNRM-ESM2-1',
                                                                                   'GFDL-CM4','GFDL-ESM4','GISS-E2-1-H', 'IPSL-CM6A-LR',
                                                                                   'MPI-ESM1-2-HR','MRI-ESM2-0','UKESM1-0-LL']})

In [ ]:
model_2300 = ['ACCESS-CM2','ACCESS-ESM1-5','CanESM5','CESM2-WACCM','GISS-E2-1-H', 'IPSL-CM6A-LR','MRI-ESM2-0','UKESM1-0-LL']

In [ ]:
bay_weights = weight_file['bay_weights']
bay_weights_2300 = weight_2300_file['bay_weights']

In [ ]:
mass_balance_weighted_yy = file_viability_info['MASS_BALANCE'].sel(time=[2000,2100]).weighted(bay_weights * sens_weights)
mass_balance_weighted_yy_2300 = file_viability_info['MASS_BALANCE'].sel(time=[2000,2100,2200,2299]).sel(model=model_2300).weighted(bay_weights_2300 * sens_weights.sel(model=model_2300))

In [ ]:
weighted_quantiles = mass_balance_weighted_yy.quantile([0,0.1,0.33,0.66,0.5,0.9],dim=['model','param','m'])
weighted_quantiles_2300 = mass_balance_weighted_yy_2300.quantile([0,0.1,0.33,0.66,0.5,0.9],dim=['model','param','m'])

In [ ]:
likelihood_list = ['vunlikely','unlikely','about','likely','vlikely']

In [ ]:
quant_distrib = (weighted_quantiles > 0).sum('quantile')
quant_distrib_2300 = (weighted_quantiles_2300 > 0).sum('quantile')

PREPARE PLOT

In [ ]:
grounded_msk03 = file_isf['ground_mask'].where(file_isf['ground_mask']==0,3)
grounded_msk = (grounded_msk03.where(grounded_msk03!=3,1)-1)*-1

In [ ]:
icesheet_msk_0inf = file_isf_mask.where(file_isf_mask!=1,0)
icesheet_msk = icesheet_msk_0inf.where(icesheet_msk_0inf < 1, 1)

In [ ]:
### Colorbar:
cmap = mpl.cm.YlOrBr # define the colormap
# extract all colors from the .jet map
cmaplist = [cmap(i) for i in range(cmap.N)]
# force the first color entry to be grey
# cmaplist[0] = (.5, .5, .5, 1.0)

# create the new map
cmap_new = mpl.colors.LinearSegmentedColormap.from_list(
    'Custom cmap', cmaplist, cmap.N)

# define the bins and normalize
bounds = np.arange(6)
norm = mpl.colors.BoundaryNorm(bounds, cmap_new.N)



In [ ]:
scen = 'ssp126'
yy = 2299
via_to_plot = quant_distrib_2300.sel(time=yy,scenario=scen)

map_to_plot = file_isf['ISF_mask'] * np.nan
for kisf in tqdm(file_isf['Nisf']):
    map_to_plot = map_to_plot.where(file_isf['ISF_mask'] != kisf, via_to_plot.sel(Nisf=kisf))
    

fig = plt.figure(figsize=(8.25,8.25))

# MAR masks:
grd=file_isf['ISF_mask']
msk_ice = grd.where( (grd>0.) )* 0 + 1

# Basin masks:
basin=xr.open_dataset('/bettik/burgardc/DATA/SUMMER_PAPER/interim/Mask_Iceshelf_4km_IMBIE_withNisf.nc')

##########################################################################
# PLOT :

#----------
# Colorbar:
#cbar_range = np.arange(0.,4200.,200.)
#newcolors = col(np.linspace(0.47, 1.00,256))

#----------
# Defining colormap:

# moving the zero of colorbar
# NB: modify the Ncool to Nwarm ratio (total=256) to place zero as desired.
#Ncool=int(256*(-np.amin(cbar_range)/(np.amax(cbar_range)-np.amin(cbar_range))))
#Nwarm=256-Ncool
#col = cm.get_cmap('PuOr', 256)
#tmp1 = col(np.linspace(0.47, 1.00, Ncool)) # decrease first number to have more white in the middle light-blue colors
#tmp2 = col(np.linspace(0.00, 0.51, Nwarm)) # increase second number to have more white in the middle light-yellow colors
#newcolors = np.append(tmp1[::-1,:],tmp2[::-1,:],axis=0)
#newcolors = col(np.linspace(0.47, 1.00,256))
#cmap_new = ListedColormap(newcolors)

#cax=fig.add_axes([0.52, 0.02, 0.43, 0.015]) # color bar
im0=plt.contourf(grd.x,grd.y,map_to_plot,extend='max',cmap=cmap_new, norm=norm)
plt.contour(grd.x,grd.y,grounded_msk,linewidths=0.5,colors='black',zorder=10)
plt.contour(grd.x,grd.y,icesheet_msk,linewidths=0.5,colors='black',zorder=15)

plt.title('Viability probability '+str(yy)+' '+scen)

ax2 = fig.add_axes([0.90, 0.1, 0.03, 0.8])
cb = mpl.colorbar.ColorbarBase(ax2, cmap=cmap_new, norm=norm,
    spacing='proportional', ticks=np.arange(0.5,5), boundaries=bounds) #, format='%1i'
cb.set_ticklabels(likelihood_list, rotation=90)

#plt.tight_layout()
fig.savefig(plot_path + 'map_2D_likelihood_'+str(yy)+'_'+scen+'.png', dpi=300)

In [ ]:
center_x_list = []
center_y_list = []
for kisf in tqdm(file_isf.Nisf):
    kisf_mask = isf_mask.where(isf_mask == kisf, drop=True)
    center_x = kisf_mask.x.isel(x=np.ceil(len(kisf_mask.x)/2).astype(int))
    center_y = kisf_mask.y.isel(y=np.ceil(len(kisf_mask.y)/2).astype(int))

    dx = kisf_mask.x.isel(x=1) - kisf_mask.x.isel(x=0)

    if np.isnan(kisf_mask.sel(x=center_x,y=center_y)):
        center_x = kisf_mask.x.min() 
        while np.isnan(kisf_mask.sel(x=center_x,y=center_y)):
            center_x = center_x + dx

    center_x_list.append(xr.DataArray(data=center_x.values).assign_coords({'Nisf': kisf}))
    center_y_list.append(xr.DataArray(data=center_y.values).assign_coords({'Nisf': kisf}))

center_dim = xr.Dataset()
center_dim['center_x'] = xr.concat(center_x_list, dim='Nisf')
center_dim['center_y'] = xr.concat(center_y_list, dim='Nisf')


In [ ]:
fig, axs = plt.subplots(nrows=1,ncols=1,figsize=(18.0,18.0))
im0=axs.contourf(grd.x,grd.y,via_to_plot,extend='max',cmap=cmap_new, norm=norm)
axs.contour(grd.x,grd.y,grounded_msk,linewidths=0.5,colors='black',zorder=10)
axs.contour(grd.x,grd.y,icesheet_msk,linewidths=0.5,colors='black',zorder=15)
axs.scatter(center_dim['center_x'],center_dim['center_y'])

In [ ]:
# barycenters of individual ice shelves:
Nbasin = basin.ID.size
xbisf = np.zeros((Nbasin)) * np.nan
ybisf = np.zeros((Nbasin)) * np.nan
xboce = np.zeros((Nbasin)) * np.nan
yboce = np.zeros((Nbasin)) * np.nan
xbint = np.zeros((Nbasin)) * np.nan # intermediate point for 2-piece segments
ybint = np.zeros((Nbasin)) * np.nan
x2d, y2d = np.meshgrid(basin.x.values, basin.y.values, indexing='xy')


for kbasin in np.arange(Nbasin):
  mskisf = ( grd.ICE_MAR - grd.GROUND ) * ( basin.Iceshelf_extrap.where( (basin.Iceshelf_extrap == basin.ID.values[kbasin]) ) * 0 + 1) * grd.af2
  mskgrd = grd.GROUND * ( basin.Iceshelf_extrap.where( (basin.Iceshelf_extrap == basin.ID.values[kbasin]) ) * 0 + 1) * grd.af2
  mskoce = ( 1. - grd.AIS ) * ( basin.Iceshelf_extrap.where( (basin.Iceshelf_extrap == basin.ID.values[kbasin]) ) * 0 + 1) * grd.af2
  tmpisf = np.nansum(mskisf.values)
  tmpgrd = np.nansum(mskgrd.values)
  tmpoce = np.nansum(mskoce.values)
  # Limit of 1500 km2 for the ice shelves and 15000 km2 for the grounded part + a few exceptions
  if ( (  ( (tmpisf*4.0**2 > 1500 ) & (tmpgrd*4.0**2 > 15000 ) ) \
        | ( basin.NAME.values[kbasin] == 'LarsenA' )   \
        | ( basin.NAME.values[kbasin] == 'LarsenB' )   \
        | ( basin.NAME.values[kbasin] == 'Wordie'  )   \
        | ( basin.NAME.values[kbasin] == 'Venable' )   \
        | ( basin.NAME.values[kbasin] == 'Cosgrove') ) \
      & ( not basin.NAME.values[kbasin] == '' ) ):
    xbisf[kbasin] = np.nansum(mskisf.values * x2d) / tmpisf
    ybisf[kbasin] = np.nansum(mskisf.values * y2d) / tmpisf
    xboce[kbasin] = np.nansum(mskoce.values * x2d) / tmpoce
    yboce[kbasin] = np.nansum(mskoce.values * y2d) / tmpoce
    xbint[kbasin] = xbisf[kbasin]
    ybint[kbasin] = ybisf[kbasin]
    # Manual corrections:
    if ( basin.NAME.values[kbasin] == 'Wordie' ):
      xbint[kbasin] = xbisf[kbasin] - 0.08e6
      xboce[kbasin] = -2.5e6
      yboce[kbasin] =  1.1e6
    elif ( basin.NAME.values[kbasin] == 'Wilkins' ):
      yboce[kbasin] = yboce[kbasin] + 0.25e6
    elif ( basin.NAME.values[kbasin] == 'George_VI' ):
      xbisf[kbasin] = -2.e6
      ybisf[kbasin] =  0.77e6
      xboce[kbasin] = -2.4e6
      yboce[kbasin] =  0.80e6
      xbint[kbasin] = xbisf[kbasin]
      ybint[kbasin] = ybisf[kbasin]
    elif ( basin.NAME.values[kbasin] == 'Venable' ):  
      xboce[kbasin] = -2.5e6 
      yboce[kbasin] = ybisf[kbasin]+0.05e6
    elif ( basin.NAME.values[kbasin] == 'Abbot' ): 
      yboce[kbasin] = ybisf[kbasin]+0.05e6
    elif ( basin.NAME.values[kbasin] == 'Cosgrove' ): 
      xbint[kbasin] = xbisf[kbasin]
      ybint[kbasin] = ybisf[kbasin] - 0.12e6
      xboce[kbasin] = -2.52e6
      yboce[kbasin] = ybint[kbasin] - 0.05e6
    elif ( basin.NAME.values[kbasin] == 'Pine_Island' ):
      xbint[kbasin] = xbisf[kbasin] - 0.05e6
      ybint[kbasin] = ybisf[kbasin] - 0.15e6
      xboce[kbasin] = xbisf[kbasin] - 0.33e6
      yboce[kbasin] = ybint[kbasin] - 0.08e6
    elif ( basin.NAME.values[kbasin] == 'Thwaites' ):
      xbisf[kbasin] = xbisf[kbasin] - 0.02e6
      xbint[kbasin] = -2.0e6
      ybint[kbasin] = -0.9e6
      xboce[kbasin] = -2.5e6
      yboce[kbasin] = ybint[kbasin] - 0.17e6
    elif ( basin.NAME.values[kbasin] == 'Dotson/Philbin_Inlet' ):
      xboce[kbasin] = xboce[kbasin] - 0.25e6
      yboce[kbasin] = yboce[kbasin] - 1.0e6
    elif ( basin.NAME.values[kbasin] == 'Crosson' ):
      xbint[kbasin] = xboce[kbasin] - 0.02e6
      ybint[kbasin] = yboce[kbasin] + 0.02e6
      xboce[kbasin] = -2.1e6
      yboce[kbasin] = -1.4e6
    elif ( basin.NAME.values[kbasin] == 'Getz' ):
      xboce[kbasin] = xbisf[kbasin] - 0.02e6
      yboce[kbasin] = yboce[kbasin] - 0.75 * (yboce[kbasin] - ybisf[kbasin])
    elif ( basin.NAME.values[kbasin] == 'Sulzberger' ):
      xboce[kbasin] = xboce[kbasin] - 2 * (xboce[kbasin] - xbisf[kbasin])
    elif ( basin.NAME.values[kbasin] == 'Ross_West' ):
      xboce[kbasin] = -0.6e6
      yboce[kbasin] = -0.5e6
    elif ( basin.NAME.values[kbasin] == 'Ross_East' ):
      xboce[kbasin] = 0.5e6
      yboce[kbasin] = -0.5e6
    elif ( basin.NAME.values[kbasin] == 'Nordenskjold/Marin/HarbordGlacier/Cheetham/GeikieInlet' ):
      xbint[kbasin] = 0.
      xboce[kbasin] = -0.45e6
      yboce[kbasin] = -1.8e6
    elif ( basin.NAME.values[kbasin] == 'Drygalski' ):
      xbint[kbasin] = xboce[kbasin]
      ybint[kbasin] = yboce[kbasin]
      xboce[kbasin] =  0.1e6
      yboce[kbasin] = -1.7e6      
    elif ( basin.NAME.values[kbasin] == 'Nansen' ):
      xbint[kbasin] = xboce[kbasin] - 0.15e6
      ybint[kbasin] = ybisf[kbasin] - 0.05e6
      xboce[kbasin] =  0.05e6
      yboce[kbasin] = -2.1e6
    elif ( basin.NAME.values[kbasin] == 'Rennick' ):
      xboce[kbasin] = xbisf[kbasin] - 0.05e6
    elif ( basin.NAME.values[kbasin] == 'Cook' ):
      xboce[kbasin] = xbisf[kbasin] - 0.05e6
      yboce[kbasin] = yboce[kbasin] - 0.13e6
    elif ( basin.NAME.values[kbasin] == 'Ninnis' ):
      xboce[kbasin] = xbisf[kbasin] - 0.05e6
      yboce[kbasin] = -2.8e6
    elif ( basin.NAME.values[kbasin] == 'Mertz' ):
      xboce[kbasin] = xbisf[kbasin] + 0.02e6
    elif ( basin.NAME.values[kbasin] == 'WattBay/Zelee/Astrolabe/Liotard/Francais/Marret/Commandant_Charcot//PourquoiPas' ):
      yboce[kbasin] = -2.9e6
    elif ( basin.NAME.values[kbasin] == 'Dibble' ):
      xboce[kbasin] = 2.4e6
    elif ( basin.NAME.values[kbasin] == 'May_Glacier/Morse/Sandford' ):
      yboce[kbasin] = yboce[kbasin] -0.08e6
    elif ( basin.NAME.values[kbasin] == 'Moscow_University' ):
      xboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Totten' ):
      xboce[kbasin] = 2.8e6
      yboce[kbasin] = ybisf[kbasin] - 0.05e6
    elif ( basin.NAME.values[kbasin] == 'Shackleton' ):
      yboce[kbasin] = ybisf[kbasin] - ( yboce[kbasin] - ybisf[kbasin] )
    elif ( basin.NAME.values[kbasin] == 'Helen' ):
      yboce[kbasin] = ybisf[kbasin] + 0.01e6
    elif ( basin.NAME.values[kbasin] == 'West' ):
      xboce[kbasin] = 3.2e6
      yboce[kbasin] = ybisf[kbasin] - 0.05e6
    elif ( basin.NAME.values[kbasin] == 'Publications' ):
      xboce[kbasin] = 2.9e6 
      yboce[kbasin] = 1.0e6 
    elif ( basin.NAME.values[kbasin] == 'Amery' ):
      xbint[kbasin] = xboce[kbasin]
      ybint[kbasin] = ybisf[kbasin] + 0.03e6
      xboce[kbasin] = 2.8e6
      yboce[kbasin] = 1.4e6
    elif ( basin.NAME.values[kbasin] == 'Utsikkar/Mulebreen/Cirque_Fjord/Hoseason/Rund_Bay' ):
      xboce[kbasin] = 2.4e6
      yboce[kbasin] = 1.8e6
    elif ( basin.NAME.values[kbasin] == 'Zubchatyy/Porter/Myers' ):
      yboce[kbasin] = yboce[kbasin] +0.2e6
    elif ( basin.NAME.values[kbasin] == 'Hannan/Telen/Skallen' ):
      xbint[kbasin] = xboce[kbasin] + 0.1e6
      ybint[kbasin] = ybint[kbasin] + 0.1e6
      xboce[kbasin] = 2.6e6
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Prince_Harald' ):
      yboce[kbasin] = yboce[kbasin] + 0.2e6
    elif ( basin.NAME.values[kbasin] == 'Baudouin' ):
      xboce[kbasin] = 1.9e6
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Borchgrevink' ):
      yboce[kbasin] = yboce[kbasin] - 0.1e6
    elif ( basin.NAME.values[kbasin] == 'Lazarev' ):
      xboce[kbasin] = xboce[kbasin] + 0.4e6
    elif ( basin.NAME.values[kbasin] == 'Nivl' ):
      ybint[kbasin] = 2.6e6
      xboce[kbasin] = xbisf[kbasin] + 0.05e6
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Vigrid' ):
      xbint[kbasin] = xbisf[kbasin] + 0.05e6
      ybint[kbasin] = 2.5e6
      xboce[kbasin] = xbint[kbasin] - 0.05e6
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Fimbul' ):
      xbisf[kbasin] = xbisf[kbasin] - 0.05e6
      xbint[kbasin] = xbint[kbasin] - 0.05e6
      xboce[kbasin] = xboce[kbasin] - 0.15e6
      ybisf[kbasin] = ybisf[kbasin]
      ybint[kbasin] = ybint[kbasin]
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Jelbart' ):
      yboce[kbasin] = yboce[kbasin] + 0.1e6
    elif ( basin.NAME.values[kbasin] == 'Ekstrom' ):
      xbint[kbasin] = xbisf[kbasin] - 0.35e6
      ybint[kbasin] = ybisf[kbasin] + 0.04e6
      xboce[kbasin] = xboce[kbasin] - 0.20e6
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Riiser-Larsen' ):
      xboce[kbasin] = xboce[kbasin] + 0.30e6
      yboce[kbasin] = yboce[kbasin] + 0.05e6
    elif ( basin.NAME.values[kbasin] == 'Brunt_Stancomb' ):
      xboce[kbasin] = xboce[kbasin] + 0.30e6
      yboce[kbasin] = yboce[kbasin] + 0.20e6
    elif ( basin.NAME.values[kbasin] == 'Dawson_Lambton/Hayes_Coats_Coast' ):
      xboce[kbasin] = xboce[kbasin] + 0.08e6
      yboce[kbasin] = yboce[kbasin] + 0.15e6
    elif ( basin.NAME.values[kbasin] == 'Filchner' ):
      xboce[kbasin] =  0.e0
      yboce[kbasin] =  1.0e6
    elif ( basin.NAME.values[kbasin] == 'Ronne' ):
      xbisf[kbasin] = xbisf[kbasin] + 0.1e6
      xbint[kbasin] = xbint[kbasin] + 0.1e6
      xboce[kbasin] = xboce[kbasin] + 0.1e6
      ybisf[kbasin] = ybisf[kbasin] + 0.1e6
      ybint[kbasin] = ybint[kbasin] + 0.1e6
    elif ( basin.NAME.values[kbasin] == 'LarsenD' ):
      xboce[kbasin] = xboce[kbasin] - 0.1e6
      yboce[kbasin] = yboce[kbasin] + 0.1e6
      xbint[kbasin] = xboce[kbasin] - 0.05e6
      ybint[kbasin] = yboce[kbasin] - 0.05e6
    elif ( basin.NAME.values[kbasin] == 'LarsenC' ):
      xboce[kbasin] = xboce[kbasin] - 0.05e6
      yboce[kbasin] = 1.8e6
    elif ( basin.NAME.values[kbasin] == 'LarsenB' ):
      xboce[kbasin] = xboce[kbasin] + 0.2e6
      yboce[kbasin] = 2.2e6
      xbint[kbasin] = xboce[kbasin] - 0.1e6
      ybint[kbasin] = yboce[kbasin] - 0.1e6
    elif ( basin.NAME.values[kbasin] == 'LarsenA' ):
      xbint[kbasin] = xbisf[kbasin]
      ybint[kbasin] = 2.60e6
      xboce[kbasin] = xboce[kbasin] + 0.2e6
      yboce[kbasin] = 2.70e6
    # Correction of names:
    if ( basin.NAME.values[kbasin] == 'Utsikkar/Mulebreen/Cirque_Fjord/Hoseason/Rund_Bay' ):
      basin.NAME.values[kbasin] = 'Utsikkar, Mulebreen, Cirque Fjord,\nHoseason, Rund Bay'
    elif ( basin.NAME.values[kbasin] == 'Nordenskjold/Marin/HarbordGlacier/Cheetham/GeikieInlet' ):
      basin.NAME.values[kbasin] = 'Nordenskjöld, Marin, Harbord,\nCheetham, Geikie Inlet'
    elif ( basin.NAME.values[kbasin] == 'WattBay/Zelee/Astrolabe/Liotard/Francais/Marret/Commandant_Charcot//PourquoiPas' ):
      basin.NAME.values[kbasin] = 'Watt Bay, Zélée, Astrolabe, Liotard, Barré,\nFrançais, Marret, Comdt Charcot, Pourquoi Pas'
    elif ( basin.NAME.values[kbasin] == 'Baudouin' ):
      basin.NAME.values[kbasin] = 'Roi Baudouin'
    elif ( basin.NAME.values[kbasin] == 'Dotson/Philbin_Inlet' ):
      basin.NAME.values[kbasin] = 'Dotson'
    elif ( basin.NAME.values[kbasin] == 'Ekstrom' ):
      basin.NAME.values[kbasin] = 'Ekström'
    elif ( basin.NAME.values[kbasin] == 'Dawson_Lambton/Hayes_Coats_Coast' ):
      basin.NAME.values[kbasin] = 'Dawson-\nLambton,\nHayes'
    basin.NAME.values[kbasin] = basin.NAME.values[kbasin].replace("_"," ")
    basin.NAME.values[kbasin] = basin.NAME.values[kbasin].replace("/",", ")
    #-
    print(basin.ID.values[kbasin], basin.NAME.values[kbasin], tmpisf*4.0**2, tmpoce*4.0**2, xboce[kbasin], yboce[kbasin])

In [ ]:
cbar0=fig.colorbar(im0, cax=cax, orientation="horizontal")
cbar0.ax.tick_params(labelsize=14)
cbar0.set_label(r'2181-2200 runoff anomaly (SSP5-8.5) w.r.t. 1995-2014 (kg m$^{-2}$ yr$^{-1}$)',labelpad=-62,fontsize=14)

axs.contour(topo.x,topo.y,topo.surface.values,[1000,2000,3000,4000],colors='darkgrey',linewidths=1)
axs.contour(topo2.x,topo2.y,topo2.mask.values,[0.5],colors='black',linewidths=1)
axs.contour(basin.x,basin.y,basin.Iceshelf,np.arange(0.5,200.5,1),colors='black',linewidths=2.5)

for kbasin in np.arange(Nbasin):

  if ( not np.isnan(xboce[kbasin]) ):

    #-----------------------------------------
    # combining the 2 ensembles:
    # 8-member ensemble if year>2100 in 16-member weighted ensemble
    for scenar in ['ssp126', 'ssp585']:
       for pct in ['05', '17', '83', '95']:
          if ( eval("year_bas_"+scenar+"_pct"+pct+"[kbasin]") == 2500 ):
             exec("year_bas_"+scenar+"_pct"+pct+"[kbasin] = np.max([2101, year_bas_"+scenar+"_pct"+pct+"b[kbasin]])")
 
    #-----------------------------------------
    # Ice Shelf Labels :

    dx = (xboce[kbasin]-xbint[kbasin]) / np.sqrt( (xboce[kbasin]-xbint[kbasin])**2 + (yboce[kbasin]-ybint[kbasin])**2 ) * 1.e4
    dy = (yboce[kbasin]-ybint[kbasin]) / np.sqrt( (xboce[kbasin]-xbint[kbasin])**2 + (yboce[kbasin]-ybint[kbasin])**2 ) * 1.e4

    if ( dx < 0. ):
      hal = 'right'
    else:
      hal = 'left'
    #--
    if ( dy < 0. ):
      val = 'top'
    else:
      val = 'bottom'
    #--
    if (  ( basin.NAME[kbasin].values == 'Nordenskjöld, Marin, Harbord,\nCheetham, Geikie Inlet' ) \
        | ( basin.NAME[kbasin].values == 'Watt Bay, Zélée, Astrolabe, Liotard, Barré,\nFrançais, Marret, Comdt Charcot, Pourquoi Pas' ) ):
      ddyy = 8.0e4
    else:
      ddyy = 0.e0
    #--
    if ( (year_bas_ssp126_pct83[kbasin] == 1500) & (year_bas_ssp126_pct17[kbasin] == 1500) ):
      textssp126='always'
    elif ( (year_bas_ssp126_pct83[kbasin] == 2500) & (year_bas_ssp126_pct17[kbasin] == 2500) ):
      textssp126='never'
    elif (year_bas_ssp126_pct83[kbasin] == 1500):
      textssp126='alw.-'+year_bas_ssp126_pct17[kbasin].astype('str')
    elif (year_bas_ssp126_pct17[kbasin] == 2500):
      textssp126=year_bas_ssp126_pct83[kbasin].astype('str')+'-nev.'
    else:
      textssp126=year_bas_ssp126_pct83[kbasin].astype('str')+'-'+year_bas_ssp126_pct17[kbasin].astype('str')
    #--
    if ( (year_bas_ssp585_pct83[kbasin] == 1500) & (year_bas_ssp585_pct17[kbasin] == 1500) ):
      textssp585='always'
    elif ( (year_bas_ssp585_pct83[kbasin] == 2500) & (year_bas_ssp585_pct17[kbasin] == 2500) ):
      textssp585='never'
    elif (year_bas_ssp585_pct83[kbasin] == 1500):
      textssp585='alw.-'+year_bas_ssp585_pct17[kbasin].astype('str')
    elif (year_bas_ssp585_pct17[kbasin] == 2500):
      textssp585=year_bas_ssp585_pct83[kbasin].astype('str')+'-nev.'
    else:
      textssp585=year_bas_ssp585_pct83[kbasin].astype('str')+'-'+year_bas_ssp585_pct17[kbasin].astype('str')
    #--
    if (year_bas_ssp126_pct83[kbasin] <= 2014):
       coltex = 'firebrick' # likely already there
    elif (year_bas_ssp126_pct83[kbasin] < 2500):
       coltex = 'red'  # may happen even under ssp126
    elif (year_bas_ssp585_pct83[kbasin] < 2100):
       coltex = 'orchid'  # likely before 2100 but only under ssp585
    elif (year_bas_ssp585_pct83[kbasin] < 2200):
       coltex = 'blueviolet'  # likely between 2100 and 2200 but only under ssp585
    else:
       coltex = 'darkblue' # unlikely before 2200
    #--
    axs.plot([xbisf[kbasin],xbint[kbasin],xboce[kbasin]],[ybisf[kbasin],ybint[kbasin],yboce[kbasin]],color=coltex)
    axs.text(xboce[kbasin]+dx,yboce[kbasin]+dy,basin.NAME.values[kbasin],fontsize=12,color=coltex,ha=hal,va=val,fontweight='bold')
    axs.text(xboce[kbasin]+dx,yboce[kbasin]+dy-8.0e4-ddyy,textssp126,fontsize=12,color=coltex,ha=hal,va=val,fontstyle='italic')
    axs.text(xboce[kbasin]+dx,yboce[kbasin]+dy-16.e4-ddyy,textssp585,fontsize=12,color=coltex,ha=hal,va=val)

# Legend for color code:
axs.text(-2.80e6,-2.3e6,'Color code :',color='black',fontsize=13)
axs.text(-2.80e6,-2.4e6,'Likely range starting before 2015',color='firebrick',fontsize=13,fontweight='bold')
axs.text(-2.80e6,-2.5e6,'Likely range reached in SSP1-2.6',color='red',fontsize=13,fontweight='bold')
axs.text(-2.80e6,-2.6e6,'Likely range starting before 2100 in SSP5-8.5',color='orchid',fontsize=13,fontweight='bold')
axs.text(-2.80e6,-2.7e6,'Likely range starting before 2200 in SSP5-8.5',color='blueviolet',fontsize=13,fontweight='bold')
axs.text(-2.80e6,-2.8e6,'Likely range not reached before 2200',color='darkblue',fontsize=13,fontweight='bold')

# Additional panel on the evolution of number of ice shelves above the threshold :
yyyy=np.arange(1850,2201,1)
Ny=np.size(yyyy)
nb_isf_ssp126_pct83 = np.zeros((Ny))
nb_isf_ssp126_pct17 = np.zeros((Ny))
nb_isf_ssp585_pct83 = np.zeros((Ny))
nb_isf_ssp585_pct17 = np.zeros((Ny))
for ky in np.arange(Ny):
   nb_isf_ssp126_pct83[ky] = np.sum( year_bas_ssp126_pct83 <= yyyy[ky] )
   nb_isf_ssp126_pct17[ky] = np.sum( year_bas_ssp126_pct17 <= yyyy[ky] )
   nb_isf_ssp585_pct83[ky] = np.sum( year_bas_ssp585_pct83 <= yyyy[ky] )
   nb_isf_ssp585_pct17[ky] = np.sum( year_bas_ssp585_pct17 <= yyyy[ky] )
llax=fig.add_axes([0.18, 0.02, 0.32, 0.10])
llax.fill_between(yyyy,nb_isf_ssp126_pct83,nb_isf_ssp126_pct17,color='cornflowerblue',alpha=0.2)
llax.fill_between(yyyy,nb_isf_ssp585_pct83,nb_isf_ssp585_pct17,color='firebrick',alpha=0.2)
llax.text(1855,120,'Number of ice shelves above threshold',fontsize=14)
llax.tick_params(axis='both', labelsize=14)

xc=np.mean(axs.get_xlim())
yu=axs.get_ylim()[1]
axs.text(xc,0.98*yu,'Likely emergence of runoff conditions necessary for hydrofracvturing',fontsize=20,fontweight='bold',ha='center')
axs.set_axis_off()

##########################################################################

In [ ]:
# Topo (just for the plot):
#topo=xr.open_dataset('/data/njourdain/DATA_ISMIP6/bedmap2_8km.nc')
#topo2=xr.open_dataset('/data/njourdain/DATA_ISMIP6/BedMachineAntarctica_2020-07-15_v02_8km.nc')

# Previously calculated years of hydrofracturing potential:
# First for the 16-model weighted ensemble until 2100 :
zyears=np.load('runoff_dates_2100.npz')
threshold = zyears['threshold']
timewin = zyears['timewin']
print('threshold = ',threshold,'kg/m2/yr ;   timewin = ',timewin,' years')
year_bas_ssp126_pct05 = np.array(zyears['year_bas_ssp126_pct05'])
year_bas_ssp126_pct17 = np.array(zyears['year_bas_ssp126_pct17'])
year_bas_ssp126_pct83 = np.array(zyears['year_bas_ssp126_pct83'])
year_bas_ssp126_pct95 = np.array(zyears['year_bas_ssp126_pct95'])
year_bas_ssp245_pct05 = np.array(zyears['year_bas_ssp245_pct05'])
year_bas_ssp245_pct17 = np.array(zyears['year_bas_ssp245_pct17'])
year_bas_ssp245_pct83 = np.array(zyears['year_bas_ssp245_pct83'])
year_bas_ssp245_pct95 = np.array(zyears['year_bas_ssp245_pct95'])
year_bas_ssp585_pct05 = np.array(zyears['year_bas_ssp585_pct05'])
year_bas_ssp585_pct17 = np.array(zyears['year_bas_ssp585_pct17'])
year_bas_ssp585_pct83 = np.array(zyears['year_bas_ssp585_pct83'])
year_bas_ssp585_pct95 = np.array(zyears['year_bas_ssp585_pct95'])
# Second for the 8-model ensemble until 2200 :
zyearsb=np.load('runoff_dates_2200.npz')
thresholdb = zyearsb['threshold']
timewinb = zyearsb['timewin']
if ( ( not threshold == thresholdb ) | ( not timewin == timewinb ) ):
  print(' *** !!! WARNIG !!! *** CHECK CONSISTENCY OF threshold AND timewin VALUES !!!!!!! ><><><><><><><><><<><><><><><<<>>>><<<<>>>>>')
year_bas_ssp126_pct05b = np.array(zyearsb['year_bas_ssp126_pct05'])
year_bas_ssp126_pct17b = np.array(zyearsb['year_bas_ssp126_pct17'])
year_bas_ssp126_pct83b = np.array(zyearsb['year_bas_ssp126_pct83'])
year_bas_ssp126_pct95b = np.array(zyearsb['year_bas_ssp126_pct95'])
year_bas_ssp585_pct05b = np.array(zyearsb['year_bas_ssp585_pct05'])
year_bas_ssp585_pct17b = np.array(zyearsb['year_bas_ssp585_pct17'])
year_bas_ssp585_pct83b = np.array(zyearsb['year_bas_ssp585_pct83'])
year_bas_ssp585_pct95b = np.array(zyearsb['year_bas_ssp585_pct95'])


model  = [ 'ACCESS-CM2', 'ACCESS-ESM1-5', 'CanESM5' , 'CESM2-WACCM', 'GISS-E2-1-H', 'IPSL-CM6A-LR',  'MRI-ESM2-0', 'UKESM1-0-LL' ]
member = [ 'r1i1p1f1'  , 'r1i1p1f1'     , 'r1i1p1f1', 'r1i1p1f1'   , 'r1i1p1f2'   , 'r1i1p1f1'    ,  'r1i1p1f1'  , 'r4i1p1f2'    ]
Nmod = np.size(model)

map_2181_2200_585 = np.zeros(np.shape(msk_ice))
nn585 = 0

############################################################

if not os.path.exists(file_z): 


   for kmod in np.arange(Nmod):
      
     print(model[kmod])
        
     file_his = 'MAR-'+model[kmod]+'_aru_1980-2014_histo_regrid_04000m.nc'
     if not os.path.exists(file_his):
       file_his = 'MAR-'+model[kmod]+'-'+member[kmod]+'_aru_1980-2014_histo_regrid_04000m_FROM_6_MODELS.nc'
     if ( model[kmod] == 'UKESM1-0-LL' ):
       file_his = 'MAR-'+model[kmod]+'-'+member[kmod]+'_aru_1980-2014_histo_regrid_04000m_FROM_UKESM1-0-LL-r1i1p1f2-histo.nc'
      
     file_ssp126 = 'MAR-'+model[kmod]+'_aru_2015-2200_ssp126_regrid_04000m.nc'
     if not os.path.exists(file_ssp126):
       file_ssp126 = 'MAR-'+model[kmod]+'_aru_2015-2200_ssp126_regrid_04000m_FROM_ssp585.nc'
       if not os.path.exists(file_ssp126):
         file_ssp126 = 'MAR-'+model[kmod]+'-'+member[kmod]+'_aru_2015-2200_ssp126_regrid_04000m_MERGED.nc'
        
     file_ssp585 = 'MAR-'+model[kmod]+'_aru_2015-2200_ssp585_regrid_04000m.nc'
     if not os.path.exists(file_ssp585):
       file_ssp585 = 'MAR-'+model[kmod]+'-'+member[kmod]+'_aru_2015-2200_ssp585_regrid_04000m_MERGED.nc'
        
     dd585=xr.open_dataset(file_ssp585,decode_cf=False)
     map_2181_2200_585 = map_2181_2200_585 + dd.aru.isel(time=slice(165,186)).mean(dim=["time"]).values
     nn585 = nn585 + 1
  
   map_2181_2200_585 = map_2181_2200_585 / nn585

   np.savez(file_z,map_2181_2200_585 = map_2181_2200_585)

else:

   zz=np.load(file_z)
   map_2181_2200 = zz['map_2181_2200']

map_2181_2200 = map_2181_2200 * msk_ice.values * 365.25 * 86400      # mm w. eq. / year

##########################################################################

# barycenters of individual ice shelves and corresponding ocean basin:
Nbasin = basin.ID.size
xbisf = np.zeros((Nbasin)) * np.nan
ybisf = np.zeros((Nbasin)) * np.nan
xboce = np.zeros((Nbasin)) * np.nan
yboce = np.zeros((Nbasin)) * np.nan
xbint = np.zeros((Nbasin)) * np.nan # intermediate point for 2-piece segments
ybint = np.zeros((Nbasin)) * np.nan
x2d, y2d = np.meshgrid(basin.x.values, basin.y.values, indexing='xy')
for kbasin in np.arange(Nbasin):
  mskisf = ( grd.ICE_MAR - grd.GROUND ) * ( basin.Iceshelf_extrap.where( (basin.Iceshelf_extrap == basin.ID.values[kbasin]) ) * 0 + 1) * grd.af2
  mskgrd = grd.GROUND * ( basin.Iceshelf_extrap.where( (basin.Iceshelf_extrap == basin.ID.values[kbasin]) ) * 0 + 1) * grd.af2
  mskoce = ( 1. - grd.AIS ) * ( basin.Iceshelf_extrap.where( (basin.Iceshelf_extrap == basin.ID.values[kbasin]) ) * 0 + 1) * grd.af2
  tmpisf = np.nansum(mskisf.values)
  tmpgrd = np.nansum(mskgrd.values)
  tmpoce = np.nansum(mskoce.values)
  # Limit of 1500 km2 for the ice shelves and 15000 km2 for the grounded part + a few exceptions
  if ( (  ( (tmpisf*4.0**2 > 1500 ) & (tmpgrd*4.0**2 > 15000 ) ) \
        | ( basin.NAME.values[kbasin] == 'LarsenA' )   \
        | ( basin.NAME.values[kbasin] == 'LarsenB' )   \
        | ( basin.NAME.values[kbasin] == 'Wordie'  )   \
        | ( basin.NAME.values[kbasin] == 'Venable' )   \
        | ( basin.NAME.values[kbasin] == 'Cosgrove') ) \
      & ( not basin.NAME.values[kbasin] == '' ) ):
    xbisf[kbasin] = np.nansum(mskisf.values * x2d) / tmpisf
    ybisf[kbasin] = np.nansum(mskisf.values * y2d) / tmpisf
    xboce[kbasin] = np.nansum(mskoce.values * x2d) / tmpoce
    yboce[kbasin] = np.nansum(mskoce.values * y2d) / tmpoce
    xbint[kbasin] = xbisf[kbasin]
    ybint[kbasin] = ybisf[kbasin]
    # Manual corrections:
    if ( basin.NAME.values[kbasin] == 'Wordie' ):
      xbint[kbasin] = xbisf[kbasin] - 0.08e6
      xboce[kbasin] = -2.5e6
      yboce[kbasin] =  1.1e6
    elif ( basin.NAME.values[kbasin] == 'Wilkins' ):
      yboce[kbasin] = yboce[kbasin] + 0.25e6
    elif ( basin.NAME.values[kbasin] == 'George_VI' ):
      xbisf[kbasin] = -2.e6
      ybisf[kbasin] =  0.77e6
      xboce[kbasin] = -2.4e6
      yboce[kbasin] =  0.80e6
      xbint[kbasin] = xbisf[kbasin]
      ybint[kbasin] = ybisf[kbasin]
    elif ( basin.NAME.values[kbasin] == 'Venable' ):  
      xboce[kbasin] = -2.5e6 
      yboce[kbasin] = ybisf[kbasin]+0.05e6
    elif ( basin.NAME.values[kbasin] == 'Abbot' ): 
      yboce[kbasin] = ybisf[kbasin]+0.05e6
    elif ( basin.NAME.values[kbasin] == 'Cosgrove' ): 
      xbint[kbasin] = xbisf[kbasin]
      ybint[kbasin] = ybisf[kbasin] - 0.12e6
      xboce[kbasin] = -2.52e6
      yboce[kbasin] = ybint[kbasin] - 0.05e6
    elif ( basin.NAME.values[kbasin] == 'Pine_Island' ):
      xbint[kbasin] = xbisf[kbasin] - 0.05e6
      ybint[kbasin] = ybisf[kbasin] - 0.15e6
      xboce[kbasin] = xbisf[kbasin] - 0.33e6
      yboce[kbasin] = ybint[kbasin] - 0.08e6
    elif ( basin.NAME.values[kbasin] == 'Thwaites' ):
      xbisf[kbasin] = xbisf[kbasin] - 0.02e6
      xbint[kbasin] = -2.0e6
      ybint[kbasin] = -0.9e6
      xboce[kbasin] = -2.5e6
      yboce[kbasin] = ybint[kbasin] - 0.17e6
    elif ( basin.NAME.values[kbasin] == 'Dotson/Philbin_Inlet' ):
      xboce[kbasin] = xboce[kbasin] - 0.25e6
      yboce[kbasin] = yboce[kbasin] - 1.0e6
    elif ( basin.NAME.values[kbasin] == 'Crosson' ):
      xbint[kbasin] = xboce[kbasin] - 0.02e6
      ybint[kbasin] = yboce[kbasin] + 0.02e6
      xboce[kbasin] = -2.1e6
      yboce[kbasin] = -1.4e6
    elif ( basin.NAME.values[kbasin] == 'Getz' ):
      xboce[kbasin] = xbisf[kbasin] - 0.02e6
      yboce[kbasin] = yboce[kbasin] - 0.75 * (yboce[kbasin] - ybisf[kbasin])
    elif ( basin.NAME.values[kbasin] == 'Sulzberger' ):
      xboce[kbasin] = xboce[kbasin] - 2 * (xboce[kbasin] - xbisf[kbasin])
    elif ( basin.NAME.values[kbasin] == 'Ross_West' ):
      xboce[kbasin] = -0.6e6
      yboce[kbasin] = -0.5e6
    elif ( basin.NAME.values[kbasin] == 'Ross_East' ):
      xboce[kbasin] = 0.5e6
      yboce[kbasin] = -0.5e6
    elif ( basin.NAME.values[kbasin] == 'Nordenskjold/Marin/HarbordGlacier/Cheetham/GeikieInlet' ):
      xbint[kbasin] = 0.
      xboce[kbasin] = -0.45e6
      yboce[kbasin] = -1.8e6
    elif ( basin.NAME.values[kbasin] == 'Drygalski' ):
      xbint[kbasin] = xboce[kbasin]
      ybint[kbasin] = yboce[kbasin]
      xboce[kbasin] =  0.1e6
      yboce[kbasin] = -1.7e6      
    elif ( basin.NAME.values[kbasin] == 'Nansen' ):
      xbint[kbasin] = xboce[kbasin] - 0.15e6
      ybint[kbasin] = ybisf[kbasin] - 0.05e6
      xboce[kbasin] =  0.05e6
      yboce[kbasin] = -2.1e6
    elif ( basin.NAME.values[kbasin] == 'Rennick' ):
      xboce[kbasin] = xbisf[kbasin] - 0.05e6
    elif ( basin.NAME.values[kbasin] == 'Cook' ):
      xboce[kbasin] = xbisf[kbasin] - 0.05e6
      yboce[kbasin] = yboce[kbasin] - 0.13e6
    elif ( basin.NAME.values[kbasin] == 'Ninnis' ):
      xboce[kbasin] = xbisf[kbasin] - 0.05e6
      yboce[kbasin] = -2.8e6
    elif ( basin.NAME.values[kbasin] == 'Mertz' ):
      xboce[kbasin] = xbisf[kbasin] + 0.02e6
    elif ( basin.NAME.values[kbasin] == 'WattBay/Zelee/Astrolabe/Liotard/Francais/Marret/Commandant_Charcot//PourquoiPas' ):
      yboce[kbasin] = -2.9e6
    elif ( basin.NAME.values[kbasin] == 'Dibble' ):
      xboce[kbasin] = 2.4e6
    elif ( basin.NAME.values[kbasin] == 'May_Glacier/Morse/Sandford' ):
      yboce[kbasin] = yboce[kbasin] -0.08e6
    elif ( basin.NAME.values[kbasin] == 'Moscow_University' ):
      xboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Totten' ):
      xboce[kbasin] = 2.8e6
      yboce[kbasin] = ybisf[kbasin] - 0.05e6
    elif ( basin.NAME.values[kbasin] == 'Shackleton' ):
      yboce[kbasin] = ybisf[kbasin] - ( yboce[kbasin] - ybisf[kbasin] )
    elif ( basin.NAME.values[kbasin] == 'Helen' ):
      yboce[kbasin] = ybisf[kbasin] + 0.01e6
    elif ( basin.NAME.values[kbasin] == 'West' ):
      xboce[kbasin] = 3.2e6
      yboce[kbasin] = ybisf[kbasin] - 0.05e6
    elif ( basin.NAME.values[kbasin] == 'Publications' ):
      xboce[kbasin] = 2.9e6 
      yboce[kbasin] = 1.0e6 
    elif ( basin.NAME.values[kbasin] == 'Amery' ):
      xbint[kbasin] = xboce[kbasin]
      ybint[kbasin] = ybisf[kbasin] + 0.03e6
      xboce[kbasin] = 2.8e6
      yboce[kbasin] = 1.4e6
    elif ( basin.NAME.values[kbasin] == 'Utsikkar/Mulebreen/Cirque_Fjord/Hoseason/Rund_Bay' ):
      xboce[kbasin] = 2.4e6
      yboce[kbasin] = 1.8e6
    elif ( basin.NAME.values[kbasin] == 'Zubchatyy/Porter/Myers' ):
      yboce[kbasin] = yboce[kbasin] +0.2e6
    elif ( basin.NAME.values[kbasin] == 'Hannan/Telen/Skallen' ):
      xbint[kbasin] = xboce[kbasin] + 0.1e6
      ybint[kbasin] = ybint[kbasin] + 0.1e6
      xboce[kbasin] = 2.6e6
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Prince_Harald' ):
      yboce[kbasin] = yboce[kbasin] + 0.2e6
    elif ( basin.NAME.values[kbasin] == 'Baudouin' ):
      xboce[kbasin] = 1.9e6
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Borchgrevink' ):
      yboce[kbasin] = yboce[kbasin] - 0.1e6
    elif ( basin.NAME.values[kbasin] == 'Lazarev' ):
      xboce[kbasin] = xboce[kbasin] + 0.4e6
    elif ( basin.NAME.values[kbasin] == 'Nivl' ):
      ybint[kbasin] = 2.6e6
      xboce[kbasin] = xbisf[kbasin] + 0.05e6
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Vigrid' ):
      xbint[kbasin] = xbisf[kbasin] + 0.05e6
      ybint[kbasin] = 2.5e6
      xboce[kbasin] = xbint[kbasin] - 0.05e6
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Fimbul' ):
      xbisf[kbasin] = xbisf[kbasin] - 0.05e6
      xbint[kbasin] = xbint[kbasin] - 0.05e6
      xboce[kbasin] = xboce[kbasin] - 0.15e6
      ybisf[kbasin] = ybisf[kbasin]
      ybint[kbasin] = ybint[kbasin]
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Jelbart' ):
      yboce[kbasin] = yboce[kbasin] + 0.1e6
    elif ( basin.NAME.values[kbasin] == 'Ekstrom' ):
      xbint[kbasin] = xbisf[kbasin] - 0.35e6
      ybint[kbasin] = ybisf[kbasin] + 0.04e6
      xboce[kbasin] = xboce[kbasin] - 0.20e6
      yboce[kbasin] = 2.7e6
    elif ( basin.NAME.values[kbasin] == 'Riiser-Larsen' ):
      xboce[kbasin] = xboce[kbasin] + 0.30e6
      yboce[kbasin] = yboce[kbasin] + 0.05e6
    elif ( basin.NAME.values[kbasin] == 'Brunt_Stancomb' ):
      xboce[kbasin] = xboce[kbasin] + 0.30e6
      yboce[kbasin] = yboce[kbasin] + 0.20e6
    elif ( basin.NAME.values[kbasin] == 'Dawson_Lambton/Hayes_Coats_Coast' ):
      xboce[kbasin] = xboce[kbasin] + 0.08e6
      yboce[kbasin] = yboce[kbasin] + 0.15e6
    elif ( basin.NAME.values[kbasin] == 'Filchner' ):
      xboce[kbasin] =  0.e0
      yboce[kbasin] =  1.0e6
    elif ( basin.NAME.values[kbasin] == 'Ronne' ):
      xbisf[kbasin] = xbisf[kbasin] + 0.1e6
      xbint[kbasin] = xbint[kbasin] + 0.1e6
      xboce[kbasin] = xboce[kbasin] + 0.1e6
      ybisf[kbasin] = ybisf[kbasin] + 0.1e6
      ybint[kbasin] = ybint[kbasin] + 0.1e6
    elif ( basin.NAME.values[kbasin] == 'LarsenD' ):
      xboce[kbasin] = xboce[kbasin] - 0.1e6
      yboce[kbasin] = yboce[kbasin] + 0.1e6
      xbint[kbasin] = xboce[kbasin] - 0.05e6
      ybint[kbasin] = yboce[kbasin] - 0.05e6
    elif ( basin.NAME.values[kbasin] == 'LarsenC' ):
      xboce[kbasin] = xboce[kbasin] - 0.05e6
      yboce[kbasin] = 1.8e6
    elif ( basin.NAME.values[kbasin] == 'LarsenB' ):
      xboce[kbasin] = xboce[kbasin] + 0.2e6
      yboce[kbasin] = 2.2e6
      xbint[kbasin] = xboce[kbasin] - 0.1e6
      ybint[kbasin] = yboce[kbasin] - 0.1e6
    elif ( basin.NAME.values[kbasin] == 'LarsenA' ):
      xbint[kbasin] = xbisf[kbasin]
      ybint[kbasin] = 2.60e6
      xboce[kbasin] = xboce[kbasin] + 0.2e6
      yboce[kbasin] = 2.70e6
    # Correction of names:
    if ( basin.NAME.values[kbasin] == 'Utsikkar/Mulebreen/Cirque_Fjord/Hoseason/Rund_Bay' ):
      basin.NAME.values[kbasin] = 'Utsikkar, Mulebreen, Cirque Fjord,\nHoseason, Rund Bay'
    elif ( basin.NAME.values[kbasin] == 'Nordenskjold/Marin/HarbordGlacier/Cheetham/GeikieInlet' ):
      basin.NAME.values[kbasin] = 'Nordenskjöld, Marin, Harbord,\nCheetham, Geikie Inlet'
    elif ( basin.NAME.values[kbasin] == 'WattBay/Zelee/Astrolabe/Liotard/Francais/Marret/Commandant_Charcot//PourquoiPas' ):
      basin.NAME.values[kbasin] = 'Watt Bay, Zélée, Astrolabe, Liotard, Barré,\nFrançais, Marret, Comdt Charcot, Pourquoi Pas'
    elif ( basin.NAME.values[kbasin] == 'Baudouin' ):
      basin.NAME.values[kbasin] = 'Roi Baudouin'
    elif ( basin.NAME.values[kbasin] == 'Dotson/Philbin_Inlet' ):
      basin.NAME.values[kbasin] = 'Dotson'
    elif ( basin.NAME.values[kbasin] == 'Ekstrom' ):
      basin.NAME.values[kbasin] = 'Ekström'
    elif ( basin.NAME.values[kbasin] == 'Dawson_Lambton/Hayes_Coats_Coast' ):
      basin.NAME.values[kbasin] = 'Dawson-\nLambton,\nHayes'
    basin.NAME.values[kbasin] = basin.NAME.values[kbasin].replace("_"," ")
    basin.NAME.values[kbasin] = basin.NAME.values[kbasin].replace("/",", ")
    #-
    print(basin.ID.values[kbasin], basin.NAME.values[kbasin], tmpisf*4.0**2, tmpoce*4.0**2, xboce[kbasin], yboce[kbasin])

In [ ]:
##########################################################################
# PLOT :

#----------
# Colorbar:
cbar_range = np.arange(0.,4200.,200.)

#----------
# Defining colormap:

# moving the zero of colorbar
# NB: modify the Ncool to Nwarm ratio (total=256) to place zero as desired.
Ncool=int(256*(-np.amin(cbar_range)/(np.amax(cbar_range)-np.amin(cbar_range))))
Nwarm=256-Ncool
col = cm.get_cmap('PuOr', 256)
#tmp1 = col(np.linspace(0.47, 1.00, Ncool)) # decrease first number to have more white in the middle light-blue colors
#tmp2 = col(np.linspace(0.00, 0.51, Nwarm)) # increase second number to have more white in the middle light-yellow colors
#newcolors = np.append(tmp1[::-1,:],tmp2[::-1,:],axis=0)
newcolors = col(np.linspace(0.47, 1.00,256))
cmap_new = ListedColormap(newcolors)

cax=fig.add_axes([0.52, 0.02, 0.43, 0.015]) # color bar
im0=axs.contourf(grd.x,grd.y,map_2181_2200,cbar_range,cmap=cmap_new,extend='max')
cbar0=fig.colorbar(im0, cax=cax, orientation="horizontal")
cbar0.ax.tick_params(labelsize=14)
cbar0.set_label(r'2181-2200 runoff anomaly (SSP5-8.5) w.r.t. 1995-2014 (kg m$^{-2}$ yr$^{-1}$)',labelpad=-62,fontsize=14)

axs.contour(topo.x,topo.y,topo.surface.values,[1000,2000,3000,4000],colors='darkgrey',linewidths=1)
axs.contour(topo2.x,topo2.y,topo2.mask.values,[0.5],colors='black',linewidths=1)
axs.contour(basin.x,basin.y,basin.Iceshelf,np.arange(0.5,200.5,1),colors='black',linewidths=2.5)

for kbasin in np.arange(Nbasin):

  if ( not np.isnan(xboce[kbasin]) ):

    #-----------------------------------------
    # combining the 2 ensembles:
    # 8-member ensemble if year>2100 in 16-member weighted ensemble
    for scenar in ['ssp126', 'ssp585']:
       for pct in ['05', '17', '83', '95']:
          if ( eval("year_bas_"+scenar+"_pct"+pct+"[kbasin]") == 2500 ):
             exec("year_bas_"+scenar+"_pct"+pct+"[kbasin] = np.max([2101, year_bas_"+scenar+"_pct"+pct+"b[kbasin]])")
 
    #-----------------------------------------
    # Ice Shelf Labels :

    dx = (xboce[kbasin]-xbint[kbasin]) / np.sqrt( (xboce[kbasin]-xbint[kbasin])**2 + (yboce[kbasin]-ybint[kbasin])**2 ) * 1.e4
    dy = (yboce[kbasin]-ybint[kbasin]) / np.sqrt( (xboce[kbasin]-xbint[kbasin])**2 + (yboce[kbasin]-ybint[kbasin])**2 ) * 1.e4

    if ( dx < 0. ):
      hal = 'right'
    else:
      hal = 'left'
    #--
    if ( dy < 0. ):
      val = 'top'
    else:
      val = 'bottom'
    #--
    if (  ( basin.NAME[kbasin].values == 'Nordenskjöld, Marin, Harbord,\nCheetham, Geikie Inlet' ) \
        | ( basin.NAME[kbasin].values == 'Watt Bay, Zélée, Astrolabe, Liotard, Barré,\nFrançais, Marret, Comdt Charcot, Pourquoi Pas' ) ):
      ddyy = 8.0e4
    else:
      ddyy = 0.e0
    #--
    if ( (year_bas_ssp126_pct83[kbasin] == 1500) & (year_bas_ssp126_pct17[kbasin] == 1500) ):
      textssp126='always'
    elif ( (year_bas_ssp126_pct83[kbasin] == 2500) & (year_bas_ssp126_pct17[kbasin] == 2500) ):
      textssp126='never'
    elif (year_bas_ssp126_pct83[kbasin] == 1500):
      textssp126='alw.-'+year_bas_ssp126_pct17[kbasin].astype('str')
    elif (year_bas_ssp126_pct17[kbasin] == 2500):
      textssp126=year_bas_ssp126_pct83[kbasin].astype('str')+'-nev.'
    else:
      textssp126=year_bas_ssp126_pct83[kbasin].astype('str')+'-'+year_bas_ssp126_pct17[kbasin].astype('str')
    #--
    if ( (year_bas_ssp585_pct83[kbasin] == 1500) & (year_bas_ssp585_pct17[kbasin] == 1500) ):
      textssp585='always'
    elif ( (year_bas_ssp585_pct83[kbasin] == 2500) & (year_bas_ssp585_pct17[kbasin] == 2500) ):
      textssp585='never'
    elif (year_bas_ssp585_pct83[kbasin] == 1500):
      textssp585='alw.-'+year_bas_ssp585_pct17[kbasin].astype('str')
    elif (year_bas_ssp585_pct17[kbasin] == 2500):
      textssp585=year_bas_ssp585_pct83[kbasin].astype('str')+'-nev.'
    else:
      textssp585=year_bas_ssp585_pct83[kbasin].astype('str')+'-'+year_bas_ssp585_pct17[kbasin].astype('str')
    #--
    if (year_bas_ssp126_pct83[kbasin] <= 2014):
       coltex = 'firebrick' # likely already there
    elif (year_bas_ssp126_pct83[kbasin] < 2500):
       coltex = 'red'  # may happen even under ssp126
    elif (year_bas_ssp585_pct83[kbasin] < 2100):
       coltex = 'orchid'  # likely before 2100 but only under ssp585
    elif (year_bas_ssp585_pct83[kbasin] < 2200):
       coltex = 'blueviolet'  # likely between 2100 and 2200 but only under ssp585
    else:
       coltex = 'darkblue' # unlikely before 2200
    #--
    axs.plot([xbisf[kbasin],xbint[kbasin],xboce[kbasin]],[ybisf[kbasin],ybint[kbasin],yboce[kbasin]],color=coltex)
    axs.text(xboce[kbasin]+dx,yboce[kbasin]+dy,basin.NAME.values[kbasin],fontsize=12,color=coltex,ha=hal,va=val,fontweight='bold')
    axs.text(xboce[kbasin]+dx,yboce[kbasin]+dy-8.0e4-ddyy,textssp126,fontsize=12,color=coltex,ha=hal,va=val,fontstyle='italic')
    axs.text(xboce[kbasin]+dx,yboce[kbasin]+dy-16.e4-ddyy,textssp585,fontsize=12,color=coltex,ha=hal,va=val)

# Legend for color code:
axs.text(-2.80e6,-2.3e6,'Color code :',color='black',fontsize=13)
axs.text(-2.80e6,-2.4e6,'Likely range starting before 2015',color='firebrick',fontsize=13,fontweight='bold')
axs.text(-2.80e6,-2.5e6,'Likely range reached in SSP1-2.6',color='red',fontsize=13,fontweight='bold')
axs.text(-2.80e6,-2.6e6,'Likely range starting before 2100 in SSP5-8.5',color='orchid',fontsize=13,fontweight='bold')
axs.text(-2.80e6,-2.7e6,'Likely range starting before 2200 in SSP5-8.5',color='blueviolet',fontsize=13,fontweight='bold')
axs.text(-2.80e6,-2.8e6,'Likely range not reached before 2200',color='darkblue',fontsize=13,fontweight='bold')

# Additional panel on the evolution of number of ice shelves above the threshold :
yyyy=np.arange(1850,2201,1)
Ny=np.size(yyyy)
nb_isf_ssp126_pct83 = np.zeros((Ny))
nb_isf_ssp126_pct17 = np.zeros((Ny))
nb_isf_ssp585_pct83 = np.zeros((Ny))
nb_isf_ssp585_pct17 = np.zeros((Ny))
for ky in np.arange(Ny):
   nb_isf_ssp126_pct83[ky] = np.sum( year_bas_ssp126_pct83 <= yyyy[ky] )
   nb_isf_ssp126_pct17[ky] = np.sum( year_bas_ssp126_pct17 <= yyyy[ky] )
   nb_isf_ssp585_pct83[ky] = np.sum( year_bas_ssp585_pct83 <= yyyy[ky] )
   nb_isf_ssp585_pct17[ky] = np.sum( year_bas_ssp585_pct17 <= yyyy[ky] )
llax=fig.add_axes([0.18, 0.02, 0.32, 0.10])
llax.fill_between(yyyy,nb_isf_ssp126_pct83,nb_isf_ssp126_pct17,color='cornflowerblue',alpha=0.2)
llax.fill_between(yyyy,nb_isf_ssp585_pct83,nb_isf_ssp585_pct17,color='firebrick',alpha=0.2)
llax.text(1855,120,'Number of ice shelves above threshold',fontsize=14)
llax.tick_params(axis='both', labelsize=14)

xc=np.mean(axs.get_xlim())
yu=axs.get_ylim()[1]
axs.text(xc,0.98*yu,'Likely emergence of runoff conditions necessary for hydrofracvturing',fontsize=20,fontweight='bold',ha='center')
axs.set_axis_off()

##########################################################################

fig.savefig("map_runoff_projection.pdf")